In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

23/09/03 19:39:38 WARN Utils: Your hostname, Yinkas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.51 instead (on interface en0)
23/09/03 19:39:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/03 19:39:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Understand the Dataset

In [2]:
#reading the json
trials_df = spark.read.json('/Users/yinka/PycharmProjects/pythonProject5/venv/dataset.json', multiLine=True)

In [3]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [4]:
trials_df.dtypes

[('ageofparticipant', 'bigint'),
 ('clinician', 'struct<branch:string,name:string,role:string>'),
 ('drug_used', 'string'),
 ('experimentenddate', 'string'),
 ('experimentstartdate', 'string'),
 ('noofhourspassedatfirstreaction', 'bigint'),
 ('result', 'struct<conclusion:string,sideeffectsonparticipant:string>')]

In [5]:
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [6]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [7]:
columns = [
 'ageofparticipant',
  'clinician.branch', 'clinician.name', 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion', 'result.sideeffectsonparticipant']

In [8]:
trials_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [9]:
from pyspark.sql import functions as fn

In [10]:
#counting null values per column
trials_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



# Cleaning

In [11]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [ ]:
# flatten the dataframe
# address null values
# rename the columns
#

In [12]:
new_trials_df = trials_df.select(columns)

In [13]:
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [14]:
# rename the column
new_trials_df = new_trials_df.withColumnsRenamed({'side effects': 'side_effects', 'no of hours passed at first reaction': 'no_of_hours_passed_at_first_reaction', 'experiment start date': 'experiment_start_date', 'age of participant': 'age_of_participant', 'experiment end date': 'experiment_end_date', 'head clinician': 'head_clinician', 'assistant role': 'assistant_role'})

In [15]:
new_trials_df.show()

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [16]:
new_trials_df.describe().show()

23/09/03 19:43:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------+-------+---------+---------+--------------------+--------------------+------------------------------+-------------+------------------------+
|summary|  ageofparticipant| branch|   name|     role|drug_used|   experimentenddate| experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+-------+------------------+-------+-------+---------+---------+--------------------+--------------------+------------------------------+-------------+------------------------+
|  count|              3586|   3586|   3586|     3477|     3586|                3586|                3586|                          3513|         3533|                    3586|
|   mean|17.507250418293363|   null|   null|     null|     null|1.618381578137200...|1.615813671834913...|             44.89097637346997|         null|                    null|
| stddev|2.3066401927555233|   null|   null|     null|     null|2.3250351904618263E9|2.2862846039555306E9|         

In [17]:
new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistant role': 'unknown'})

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `assistant role` cannot be resolved. Did you mean one of the following? [`ageofparticipant`, `branch`, `name`, `role`, `drug_used`, `experimentenddate`, `experimentstartdate`, `noofhourspassedatfirstreaction`, `conclusion`, `sideeffectsonparticipant`].

In [18]:
new_trials_df.describe().show()

+-------+------------------+-------+-------+---------+---------+--------------------+--------------------+------------------------------+-------------+------------------------+
|summary|  ageofparticipant| branch|   name|     role|drug_used|   experimentenddate| experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+-------+------------------+-------+-------+---------+---------+--------------------+--------------------+------------------------------+-------------+------------------------+
|  count|              3586|   3586|   3586|     3477|     3586|                3586|                3586|                          3513|         3533|                    3586|
|   mean|17.507250418293363|   null|   null|     null|     null|1.618381578137200...|1.615813671834913...|             44.89097637346997|         null|                    null|
| stddev|2.3066401927555233|   null|   null|     null|     null|2.3250351904618263E9|2.2862846039555306E9|         

In [19]:
new_trials_df.show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

## Transformation

In [ ]:
#transforming experiment start and end date to datetype

In [20]:
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [21]:
from pyspark.sql import types as dtypes

In [ ]:
# convert the datatypes to integers (Long)
# divide by 1000
# convert from unix to datetime

In [34]:
new_trials_df = new_trials_df\
 .withColumn('start_ts', fn.from_unixtime(fn.col('experimentstartdate').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
 .withColumn('start_ts', fn.col('start_ts').cast(dtypes.TimestampType()))\
  .withColumn('end_ts', fn.from_unixtime(fn.col('experimentenddate').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
  .withColumn('end_ts', fn.col('end_ts').cast(dtypes.TimestampType()))

In [35]:
new_trials_df.show()

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+-------------------+-------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|           start_ts|             end_ts|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+-------------------+-------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|2021-03-31 18:00:00|2021-04-30 18:00:00|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|2021-03-31 18:00:00|2021-0

In [36]:
new_trials_df = new_trials_df.withColumnsRenamed({'sideeffectsonparticipant': 'side_effects', 'noofhourspassedatfirstreaction': 'no_of_hours_passed_at_first_reaction', 'experimentstartdate': 'experiment_start_date', 'ageofparticipant': 'age_of_participant', 'experimentenddate': 'experiment_end_date', 'name': 'head_clinician', 'role': 'assistant_role'})

In [38]:
new_trials_df.dtypes

[('age_of_participant', 'bigint'),
 ('branch', 'string'),
 ('head_clinician', 'string'),
 ('assistant_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('no_of_hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('side_effects', 'string'),
 ('start_ts', 'timestamp'),
 ('end_ts', 'timestamp')]

# Load (for clinicians)

In [42]:
reordered_columns = ['start_ts', 'end_ts', 'branch', 'head_clinician', 'assistant_role', 'drug_used', 'age_of_participant', 'no_of_hours_passed_at_first_reaction', 'conclusion', 'side_effects']
new_trials_df.select(reordered_columns).sort('start_ts').write.option('header', True).partitionBy('drug_used').mode('overwrite').format('csv').save('clinician')
